<a href="https://colab.research.google.com/github/areias/viral-escape/blob/main/train_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/MyDrive/hiv/load_hiv_data.py /content

In [ ]:
! pip install anndata scanpy bio

     |████████████████████████████████| 96 kB 2.5 MB/s 
     |████████████████████████████████| 2.0 MB 9.3 MB/s 
     |████████████████████████████████| 269 kB 49.3 MB/s 
     |████████████████████████████████| 88 kB 4.7 MB/s 
     |████████████████████████████████| 11.2 MB 40.4 MB/s 
     |████████████████████████████████| 930 kB 38.6 MB/s 
     |████████████████████████████████| 1.1 MB 56.3 MB/s 
     |████████████████████████████████| 2.3 MB 41.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=1140ff26b463a60dad8d4f04e9b6bec3cf347e229ee01341bf82e649dfe91036
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=c5eb7d12faacfeef7f337411d98cb80cf9f2500efdce70931687fef518501c97
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd

In [ ]:
import sys
sys.path.append('drive/MyDrive/viral-mutation/bin')

In [ ]:
import load_hiv_data 

In [ ]:
import importlib
importlib.reload(load_hiv_data)

<module 'load_hiv_data' from '/content/load_hiv_data.py'>

In [ ]:
seqs = load_hiv_data.load_hiv_data(n=1000)

In [ ]:
len(seqs)

1000

## Train Roberta 

In [46]:

! pip install tokenizers

     |████████████████████████████████| 4.2 MB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 36.8 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [40]:
def split_seqs(seqs, train_share=0.66):
    train_seqs, test_seqs = {}, {}

    print('Splitting seqs...')

    import random 
    k = int(train_share*len(seqs))
    train_idx = random.sample(range(len(seqs)),k)


    for idx, seq in enumerate(seqs):
        # Pick train set 
        if idx in train_idx:    
            train_seqs[seq] = seqs[seq]
        else:
            test_seqs[seq] = seqs[seq]

    print('{} train seqs, {} test seqs.'
           .format(len(train_seqs), len(test_seqs)))

    return train_seqs, test_seqs

In [41]:
0.5*len(seqs)

500.0

In [43]:
train_seqs, test_seqs = split_seqs(seqs,0.66)

Splitting seqs...
660 train seqs, 340 test seqs.


In [52]:
# make pretraining_data.txt file
for x,seq in enumerate(list(train_seqs.keys())):
    with open("drive/MyDrive/hiv/pretraining_data.txt", 'a') as filehandle:
        filehandle.write('%s\n' % seq)

# make eval_data.txt file
for x,seq in enumerate(list(test_seqs.keys())):
    with open("drive/MyDrive/hiv/eval_data.txt", 'a') as filehandle:
        filehandle.write('%s\n' % seq)

In [70]:
import os

paths = ['drive/MyDrive/hiv/pretraining_data', 'drive/MyDrive/hiv/eval_data','drive/MyDrive/hiv/tokenizer']
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)
        print(path + " created")

In [57]:
seq_path = 'drive/MyDrive/hiv/pretraining_data/'

for x,seq in enumerate(list(train_seqs.keys())):
    with open(seq_path+str(x)+".txt", 'w') as filehandle:
        filehandle.write('%s\n' % seq)

In [47]:
# train tokenizer 
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())

In [48]:
AAs = [
        'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
        'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W',
        'Y', 'V', 'X', 'Z', 'J', 'U', 'B',
    ]
vocabulary = { aa: idx + 1 for idx, aa in enumerate(sorted(AAs)) }

In [49]:
# hyper paramenter vocab_size
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(special_tokens=["<s>", 
                                     "</s>",
                                     "<pad>",
                                "<unk>",
                                "<mask>"],
                     initial_alphabet=list(vocabulary.keys()),
                     vocab_size=10_000)

In [58]:
from pathlib import Path
paths = [str(x) for x in Path("drive/MyDrive/hiv/pretraining_data").glob("**/*.txt")]

In [59]:
paths[:10]

['drive/MyDrive/hiv/pretraining_data/0.txt',
 'drive/MyDrive/hiv/pretraining_data/1.txt',
 'drive/MyDrive/hiv/pretraining_data/2.txt',
 'drive/MyDrive/hiv/pretraining_data/3.txt',
 'drive/MyDrive/hiv/pretraining_data/4.txt',
 'drive/MyDrive/hiv/pretraining_data/5.txt',
 'drive/MyDrive/hiv/pretraining_data/6.txt',
 'drive/MyDrive/hiv/pretraining_data/7.txt',
 'drive/MyDrive/hiv/pretraining_data/8.txt',
 'drive/MyDrive/hiv/pretraining_data/9.txt']

In [61]:
seq_path = 'drive/MyDrive/hiv/eval_data/'

for x,seq in enumerate(list(test_seqs.keys())):
    with open(seq_path+str(x)+".txt", 'w') as filehandle:
        filehandle.write('%s\n' % seq)

In [62]:
tokenizer.train(paths, trainer)

In [63]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(single="<s> $A </s>",
                                              special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ]
)

In [64]:
# hyperparameter tokenizer max_length
tokenizer.enable_truncation(max_length=512)
tokenizer.enable_padding(pad_token="<pad>")

In [66]:
tokenizer.save("drive/MyDrive/hiv/tokenizer/tokenizer.json")
tokenizer.model.save("drive/MyDrive/hiv/tokenizer")

['drive/MyDrive/hiv/tokenizer/vocab.json',
 'drive/MyDrive/hiv/tokenizer/merges.txt']

In [75]:
def prep_data_files(train_seqs, test_seqs):
    # make train_data.txt file
    for x,seq in enumerate(list(train_seqs.keys())):
        with open("drive/MyDrive/hiv/pretraining_data.txt", 'a') as filehandle:
            filehandle.write('%s\n' % seq)

    # make eval_data.txt file
    for x,seq in enumerate(list(test_seqs.keys())):
        with open("drive/MyDrive/hiv/eval_data.txt", 'a') as filehandle:
            filehandle.write('%s\n' % seq)

    import os
    # check paths if do not exists
    paths = ['drive/MyDrive/hiv/pretraining_data', 'drive/MyDrive/hiv/eval_data']
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)
            print(path + " created")

    # prep training data directory for tokenizer
    seq_path = 'drive/MyDrive/hiv/pretraining_data/'
    for x,seq in enumerate(list(train_seqs.keys())):
        with open(seq_path+str(x)+".txt", 'w') as filehandle:
            filehandle.write('%s\n' % seq)
    print("training data files saved to " + seq_path)
    
    # pret test data directory for tokenizer
    seq_path = 'drive/MyDrive/hiv/eval_data/'
    for x,seq in enumerate(list(test_seqs.keys())):
        with open(seq_path+str(x)+".txt", 'w') as filehandle:
            filehandle.write('%s\n' % seq)
    print("test data files saved to " + seq_path)



In [76]:
prep_data_files(train_seqs, test_seqs)

training data files saved to drive/MyDrive/hiv/pretraining_data/
test data files saved to drive/MyDrive/hiv/eval_data/


In [86]:
def train_tokenizer(train_seqs, test_seqs, vocab_size=10_000, max_length=512):

    # prep data files
    prep_data_files(train_seqs, test_seqs)

    # load tokenizer libraries  
    from tokenizers import Tokenizer
    from tokenizers.models import BPE

    # initialize tokenizer
    tokenizer = Tokenizer(BPE())
    
    # define vocabulary
    AAs = [
        'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
        'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W',
        'Y', 'V', 'X', 'Z', 'J', 'U', 'B',
    ]
    vocabulary = { aa: idx + 1 for idx, aa in enumerate(sorted(AAs)) }

    # hyper paramenter vocab_size
    from tokenizers.trainers import BpeTrainer
    trainer = BpeTrainer(special_tokens=["<s>", 
                                        "</s>",
                                        "<pad>",
                                    "<unk>",
                                    "<mask>"],
                        initial_alphabet=list(vocabulary.keys()),
                        vocab_size=vocab_size)
    
    # directory for training data files
    from pathlib import Path
    paths = [str(x) for x in Path("drive/MyDrive/hiv/pretraining_data").glob("**/*.txt")]
    tokenizer.train(paths, trainer)

    # post processing
    from tokenizers.processors import TemplateProcessing

    tokenizer.post_processor = TemplateProcessing(single="<s> $A </s>",
                                              special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
        ]
    )
    tokenizer.enable_truncation(max_length=max_length)
    tokenizer.enable_padding(pad_token="<pad>")
    
    import os
    # check paths if do not exists
    paths = ['drive/MyDrive/hiv/tokenizer', "drive/MyDrive/hiv/tokenizer/tokenizer_{}maxlength_{}vocabsize".format(max_length, vocab_size)]
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)
            print(path + " created")


    # save tokenizer model and dictionary
    tokenizer.save("drive/MyDrive/hiv/tokenizer/tokenizer_{}maxlength_{}vocabsize.json".format(max_length, vocab_size))
    tokenizer.model.save("drive/MyDrive/hiv/tokenizer/tokenizer_{}maxlength_{}vocabsize".format(max_length, vocab_size))
    print("tokenizer model and dictionary with {} max_length and {} vocab_size saved".format(max_length, vocab_size))

In [87]:
train_tokenizer(train_seqs, test_seqs, vocab_size=10_000, max_length=512)

training data files saved to drive/MyDrive/hiv/pretraining_data/
test data files saved to drive/MyDrive/hiv/eval_data/
drive/MyDrive/hiv/tokenizer/tokenizer_512maxlength_10000vocabsize created
tokenizer model and dictionary with 512 max_length and 10000 vocab_size saved
